In [1]:
def get_pay_off_period_variation(State,Major_Category,Debt):
    import pandas as pd
    import pprint as pp
    import pymongo

#     State = "Alaska"
#     Major_Category = "Agriculture & Natural Resources"
#     Debt = 50000


    # Create connection variable
    conn = 'mongodb://localhost:27017'

    # Pass connection to the pymongo instance.
    dbconn = pymongo.MongoClient(conn)

    # Connect to a database. Will create one if not already available.
    db = dbconn.FortunEd

    Major_data = db.Majors.find_one({"Major_Category":Major_Category})

    Income_dict = {}
    for data in Major_data["Possible_Occupations"]:
        Occ = data["Occupation"]
        income_split =data["Median_Occ_Salary"].split(",")
        income = ''.join(map(str, income_split))
    #     low_25_income_split =data['Low_25_Occ_Salary'].split(",")
    #     low_25_pct_income = ''.join(map(str, low_25_income_split))
    #     recommended_ed = data['Recommended_Education']
    #     high_25_income_split = data['High_25_Occ_Salary'].split(",")
    #     high_25_pct_income =''.join(map(str, high_25_income_split)) 
        Income_dict[Occ] ={"Occ":Occ,
                          "median_income":int(income)
    #                       "low_25_pct_income":int(low_25_pct_income),
    #                       "high_25_pct_income":int(high_25_pct_income),
    #                       "recommended_education":recommended_ed
                          }


    state_wages = db.StateWage

    state_wage_list = []
    state_wage_list = list(state_wages.find())
    result_values = [i[State] for i in state_wage_list if State in i]
    result =result_values[0]
    living_wage_split = result['living wage'].split("$")[1].split(",")
    wage_join =pd.to_numeric(''.join(map(str, living_wage_split)),errors='coerce')  



    output_df = pd.DataFrame(Income_dict.values())
    # output_df.columns = ["Occupation","Median_Income","Low_25_pct_income","High_25_pct_income","Recommended_Education"]
    output_df.columns = ["Occupation","Median_Income"]
    output_df["debt"] = Debt
    output_df["living_wage"] = wage_join
    output_df["major_category"] = Major_Category
    output_df["state"] = State
    output_df["baseline_percent"] = 30
    output_df["low_percent"] = 20
    output_df["high_percent"] = 50

    output_df["time_to_repay_baseline"] = round(output_df["debt"]/((output_df["Median_Income"] - output_df["living_wage"])*output_df["baseline_percent"]/100))
    output_df["time_to_repay_low"] = round(output_df["debt"]/((output_df["Median_Income"] - output_df["living_wage"])*output_df["low_percent"]/100))
    output_df["time_to_repay_high"] = round(output_df["debt"]/((output_df["Median_Income"] - output_df["living_wage"])*output_df["high_percent"]/100))

    outcome = output_df.to_dict("records")
    return outcome


In [2]:
get_pay_off_period_variation("Alaska","Agriculture & Natural Resources",50000)

[{'Occupation': 'First-Line Supervisors of Farming, Fishing, and Forestry Workers',
  'Median_Income': 48280,
  'debt': 50000,
  'living_wage': 26528,
  'major_category': 'Agriculture & Natural Resources',
  'state': 'Alaska',
  'baseline_percent': 30,
  'low_percent': 20,
  'high_percent': 50,
  'time_to_repay_baseline': 8.0,
  'time_to_repay_low': 11.0,
  'time_to_repay_high': 5.0},
 {'Occupation': 'Agricultural Inspectors',
  'Median_Income': 45490,
  'debt': 50000,
  'living_wage': 26528,
  'major_category': 'Agriculture & Natural Resources',
  'state': 'Alaska',
  'baseline_percent': 30,
  'low_percent': 20,
  'high_percent': 50,
  'time_to_repay_baseline': 9.0,
  'time_to_repay_low': 13.0,
  'time_to_repay_high': 5.0},
 {'Occupation': 'Animal Breeders',
  'Median_Income': 42920,
  'debt': 50000,
  'living_wage': 26528,
  'major_category': 'Agriculture & Natural Resources',
  'state': 'Alaska',
  'baseline_percent': 30,
  'low_percent': 20,
  'high_percent': 50,
  'time_to_repay_b